In [12]:
import torch
from torch_geometric.datasets import QM9
from torch_geometric.loader import DataLoader
from sklearn.model_selection import train_test_split

# 1. データセットのロードとダウンロード (初回のみ)
dataset = QM9(root='./data/QM9')

# 2. 予測ターゲットの選択
# QM9には12個のターゲットがあります。ここではターゲット G (Highest Occupied Molecular Orbital energy) を予測対象とします。
# ターゲット G はインデックス 7 です。
# ターゲット値は y[:, index] で取り出せます。

# 3. データのシャッフルと分割 (トレーニング/検証/テスト)
torch.manual_seed(42) # 再現性のためのシード設定
dataset = dataset.shuffle()

# データをインデックスで分割
train_idx, temp_idx = train_test_split(range(len(dataset)), test_size=0.2, random_state=42)
val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, random_state=42)

train_dataset = dataset[train_idx]
val_dataset = dataset[val_idx]
test_dataset = dataset[test_idx]

print(f"✅ データセットの準備完了。")
print(f"トレーニングデータ数: {len(train_dataset)}")
print(f"検証データ数: {len(val_dataset)}")
print(f"テストデータ数: {len(test_idx)}")

import torch
import torch.nn.functional as F
from torch_geometric.loader import DataLoader
from torch.nn import Linear
from torch_geometric.datasets import QM9

# 既存のデータセットをロード（ダウンロード済みなので高速）
dataset = QM9(root='./data/QM9')
# 前のステップで分割したデータセットのインデックスを使用
# train_idx, val_idx, test_idx は前段で定義済みと仮定

# データローダーの定義
BATCH_SIZE = 64
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# モデルと訓練の共通設定
TARGET_INDEX = 7 # HOMOエネルギー (インデックスG) を予測
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_node_features = dataset.num_node_features # 11
hidden_channels = 64
EPOCHS = 50 # 時間があれば増やしてください

# 評価関数: 回帰タスクでは平均絶対誤差 (MAE) を使用
criterion = torch.nn.L1Loss() 



✅ データセットの準備完了。
トレーニングデータ数: 104664
検証データ数: 13083
テストデータ数: 13084


In [13]:
def evaluate(loader, model):
    """検証・テストデータでの平均絶対誤差 (MAE) を計算する"""
    model.eval()
    total_error = 0
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            target = data.y[:, TARGET_INDEX].unsqueeze(1)
            out = model(data)
            
            # L1Lossを合計
            loss = criterion(out, target) 
            total_error += loss.item() * data.num_graphs
    return total_error / len(loader.dataset)

def train_model(model, optimizer, name):
    """汎用訓練ループ"""
    print(f"\n===== 訓練開始: {name} =====")
    model.to(device)
    best_val_mae = float('inf')
    
    for epoch in range(1, EPOCHS + 1):
        model.train()
        total_loss = 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            
            target = data.y[:, TARGET_INDEX].unsqueeze(1)
            out = model(data)
            
            loss = criterion(out, target)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * data.num_graphs
            
        train_mae = total_loss / len(train_loader.dataset)
        val_mae = evaluate(val_loader, model)
        
        if val_mae < best_val_mae:
            best_val_mae = val_mae
            # 訓練を短縮するため、ここではモデルの保存は省略
            
        print(f'Epoch: {epoch:02d}/{EPOCHS}, Train MAE: {train_mae:.4f}, Val MAE: {val_mae:.4f} (Best: {best_val_mae:.4f})')
        
    test_mae = evaluate(test_loader, model)
    print(f"=================================")
    print(f"✅ {name} のテスト MAE: {test_mae:.4f}")
    return test_mae

In [14]:
from torch_geometric.nn import GINConv, global_add_pool

class GIN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super().__init__()
        # GINConvには内部でLinear層を持つMLPを渡します
        self.conv1 = GINConv(Linear(in_channels, hidden_channels))
        self.conv2 = GINConv(Linear(hidden_channels, hidden_channels))
        self.conv3 = GINConv(Linear(hidden_channels, hidden_channels))
        
        # 最終層
        self.lin1 = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, 1) 
        
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = F.relu(self.conv3(x, edge_index))
        
        # Graph Pooling: グラフレベルの集約 (全ノード特徴量の和)
        x = global_add_pool(x, batch) 
        
        # 回帰ヘッド
        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        return self.lin2(x)

# 訓練の実行
model_gin = GIN(num_node_features, hidden_channels)
optimizer_gin = torch.optim.Adam(model_gin.parameters(), lr=0.001)
train_model(model_gin, optimizer_gin, "GIN")


===== 訓練開始: GIN =====
Epoch: 01/50, Train MAE: 2371.4824, Val MAE: 1042.1028 (Best: 1042.1028)
Epoch: 02/50, Train MAE: 1928.9767, Val MAE: 699.9485 (Best: 699.9485)
Epoch: 03/50, Train MAE: 1818.6141, Val MAE: 1358.8941 (Best: 699.9485)
Epoch: 04/50, Train MAE: 1781.1267, Val MAE: 451.4502 (Best: 451.4502)


KeyboardInterrupt: 

In [ ]:
from torch_geometric.nn import PNAConv, global_add_pool
from torch_geometric.utils import degree
import numpy as np

# PNAConvに必要な次数統計量を計算
# ノードの次数を計算し、平均と標準偏差を求める
max_deg = -1
for data in dataset:
    max_deg = max(max_deg, int(degree(data.edge_index[0]).max().item()))

# 次数ヒストグラムを作成
deg = torch.zeros(max_deg + 1, dtype=torch.long)
for data in dataset:
    d = degree(data.edge_index[0], dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())

# 次数統計
mean_deg = (deg.view(-1).float() * torch.arange(deg.numel()).float()).sum() / deg.sum()
std_deg = torch.sqrt(((deg.view(-1).float() * (torch.arange(deg.numel()).float() - mean_deg) ** 2).sum() / deg.sum()))

class PNA(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super().__init__()
        
        # PNA Convの集約/スケーリング関数のリスト
        aggregators = ['mean', 'min', 'max', 'std']
        scalers = ['identity', 'amplification', 'attenuation']
        
        # PNA層 (集合演算とMLPを組み合わせる)
        self.conv1 = PNAConv(in_channels, hidden_channels, 
                             aggregators=aggregators, scalers=scalers, 
                             deg=deg, # 次数統計を渡す
                             towers=4, # 独立した MLP の数
                             divide_input=False)
        self.conv2 = PNAConv(hidden_channels, hidden_channels, aggregators=aggregators, scalers=scalers, deg=deg)
        self.conv3 = PNAConv(hidden_channels, hidden_channels, aggregators=aggregators, scalers=scalers, deg=deg)
        
        # 最終層
        self.lin1 = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = F.relu(self.conv3(x, edge_index))
        
        x = global_add_pool(x, batch) 
        
        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        return self.lin2(x)

# 訓練の実行
model_pna = PNA(num_node_features, hidden_channels)
optimizer_pna = torch.optim.Adam(model_pna.parameters(), lr=0.001)
train_model(model_pna, optimizer_pna, "PNA")

In [ ]:
from torch_geometric.nn import GATv2Conv, global_add_pool

class GATv2(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, heads=4):
        super().__init__()
        
        # GATv2Conv層: 複数アテンションヘッド (heads=4) を使用
        # hidden_channels * heads の次元が出力されます
        self.conv1 = GATv2Conv(in_channels, hidden_channels, heads=heads) 
        self.conv2 = GATv2Conv(hidden_channels * heads, hidden_channels, heads=heads)
        
        # 最後のGATv2層: 出力次元を hidden_channels に戻し、次の層に渡す
        self.conv3 = GATv2Conv(hidden_channels * heads, hidden_channels, heads=1) 

        # 最終層
        self.lin1 = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        x = F.elu(self.conv1(x, edge_index))
        x = F.elu(self.conv2(x, edge_index))
        x = F.elu(self.conv3(x, edge_index))
        
        x = global_add_pool(x, batch) 
        
        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        return self.lin2(x)

# 訓練の実行
model_gatv2 = GATv2(num_node_features, hidden_channels, heads=4)
optimizer_gatv2 = torch.optim.Adam(model_gatv2.parameters(), lr=0.001)
train_model(model_gatv2, optimizer_gatv2, "GATv2")